In [74]:
CLS_PATHS = {
    "Empirical": "../data/results_icml_revision/classification_nn_empirical.tsv",
    "Graph": "../data/results_icml_revision/classification_nn_graph.tsv",
    "Gaussian": "../data/results_icml_revision/classification_nn_multiple_curvatures.tsv",
}

REG_PATHS = {
    "Empirical": "../data/results_icml_revision/regression_nn_empirical.tsv",
    "Graph": "../data/results_icml_revision/regression_nn_graph.tsv",
    "Gaussian": "../data/results_icml_revision/regression_nn_multiple_curvatures.tsv",
}

HIGHDIM_PATHS = {
    "Gaussian": "../data/results_icml_revision/classification_nn_single_curvature_16dim.tsv"
}

# PATHS = CLS_PATHS
# VAR = "f1-macro"

# PATHS = CLS_PATHS
# VAR = "accuracy"

# PATHS = REG_PATHS
# VAR = "mse"

PATHS = HIGHDIM_PATHS
VAR = "accuracy"


In [79]:
import pandas as pd
import numpy as np


def aggfunc(x):
    # Get 95% CI on percentages
    mean = np.mean(x)
    ci = 1.96 * np.std(x) / np.sqrt(len(x))
    if VAR in ["accuracy", "f1-macro"]:
        mean, ci = mean * 100, ci * 100
    if mean < 1e3:
        return f"{mean:.2f} ± {ci:.2f}"
    else:
        return f"{mean:.2e} ± {ci:.2e}"


def stylefunc(x):
    is_max = pd.Series(data=False, index=x.index)
    means = x.str.split("±").str[0].astype(float)
    is_max[means.idxmax()] = True if PATHS == CLS_PATHS else False
    is_max[means.idxmax()] = True if PATHS == HIGHDIM_PATHS else False
    is_max[means.idxmin()] = True if PATHS == REG_PATHS else False
    return ["background-color: lightgreen" if v else "" for v in is_max]


df = []
for name, path in PATHS.items():
    df_path = pd.read_table(path)
    df_path["type"] = name
    df_path = df_path[[c for c in df_path.columns if VAR in c or c in ["type", "dataset", "signature"]]]
    df_path.columns = [c.replace(f"_{VAR}", "") for c in df_path.columns]
    df.append(df_path)

df = pd.concat(df)
grouped_df = df.groupby(["type", "dataset", "signature"]).agg(aggfunc)
grouped_df.style.apply(stylefunc, axis=1)

In [82]:
# Cell just for saving the styled dataframe as a figure
import matplotlib.pyplot as plt
import numpy as np

# Create a numerical version by extracting the mean values
numerical_df = grouped_df.copy()
for col in numerical_df.columns:
    numerical_df[col] = numerical_df[col].str.split("±").str[0].astype(float)

# Find max/min values per row for highlighting
if PATHS in [CLS_PATHS, HIGHDIM_PATHS]:
    # For classification, highlight max values
    highlight_values = numerical_df.idxmax(axis=1)
else:
    # For regression, highlight min values
    highlight_values = numerical_df.idxmin(axis=1)

is_highlight = pd.DataFrame(False, index=numerical_df.index, columns=numerical_df.columns)
for idx, col in zip(highlight_values.index, highlight_values.values):
    is_highlight.loc[idx, col] = True

# Calculate adaptive figure dimensions
# Base width on number of columns and max text length
col_widths = [len(str(c)) + 10 for c in grouped_df.columns]  # Add padding for the values
col_count = len(grouped_df.columns)

# Get max length of row labels (might be multi-index)
if isinstance(grouped_df.index, pd.MultiIndex):
    row_label_width = max(len(", ".join(str(x) for x in i)) for i in grouped_df.index)
else:
    row_label_width = max(len(str(i)) for i in grouped_df.index)

# Calculate total width needed (in inches)
total_width = (sum(col_widths) * 0.1) + (row_label_width * 0.12)
# Set a minimum width
total_width = max(total_width, 10)

# Height based on row count (adjust based on your data size)
row_height = 0.4
total_height = max(len(grouped_df) * row_height, 5)

# Create figure with adaptive dimensions
plt.figure(figsize=(total_width, total_height))
ax = plt.subplot(111, frame_on=False)

# Hide axes
ax.xaxis.set_visible(False)
ax.yaxis.set_visible(False)

# Create the table
table = ax.table(
    cellText=grouped_df.values,
    rowLabels=grouped_df.index,
    colLabels=grouped_df.columns,
    cellLoc="center",
    loc="center",
    bbox=[0, 0, 1, 1],
)

# Style the table
table.auto_set_font_size(False)
table.set_fontsize(10)

# Apply highlighting
for i in range(len(grouped_df.index)):
    for j in range(len(grouped_df.columns)):
        if is_highlight.iloc[i, j]:
            table[(i + 1, j)].set_facecolor("lightgreen")

# Save the figure
plt.title(f"Table of {VAR} values")
plt.tight_layout()
SUFFIX = "_highdim" if PATHS == HIGHDIM_PATHS else ""
plt.savefig(f"df_{VAR}{SUFFIX}.png", dpi=300, bbox_inches="tight")
plt.close()

# print(f"Table saved as '{VAR}_table.png'")